# Download SST from Google Earth Engine
#### Load libraries, initialize EE connection

In [1]:
import ee
import geemap
import eeconvert
import pandas as pd
import geopandas as gpd
import numpy as np
ee.Initialize()

#### Define parameters for data 

In [2]:
# temporal range
start_year = 1999
end_year = 2019

# spatial extent to define region of interest
xmin = 120
xmax = 290
ymin = -30
ymax = 30

# desired resolution for sampling raster (in meters)
res_sampling = 10000

# desired resolution for point grid (in degrees)
res_grid = 2

# path to save output
file_path = '/Users/rebeccawillison/Documents/research/sst/sst_output.csv'

#### Create map and visualization palette

In [3]:
ee_ROI = ee.Geometry.Rectangle([xmin, ymin, xmax, ymax], geodesic = False, proj = 'EPSG:4326')
visParams = {
  'min': -300,
  'max': 300,
  'palette': [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003'
  ],
};
Map = geemap.Map()
Map.centerObject(ee_ROI, 3)

#### Define some functions 

In [4]:
def clipToROI(image):
    return image.clip(ee_ROI)

def summarizeByMonth(y):
    def monthlyMean(m):
        return sst_daily.filter(ee.Filter.calendarRange(y, y, 'year')) \
                     .filter(ee.Filter.calendarRange(m, m, 'month')) \
                     .mean().set('month', m).set('year', y)
    return months.map(monthlyMean)

#### Get data!

In [5]:
months = ee.List.sequence(1, 12)
years = ee.List.sequence(start_year, end_year)

sst_daily = ee.ImageCollection('NOAA/CDR/OISST/V2_1') \
           .filter(ee.Filter.date(str(start_year)+'-01-01', str(end_year+1)+'-01-01')) \
           .select('anom') \
           .filterBounds(ee_ROI) \
           .map(clipToROI)

sst = ee.ImageCollection.fromImages(years.map(summarizeByMonth).flatten())

Check the first image to make things look reasonable:

In [6]:
Map.addLayer(sst.first(), visParams, 'First SST Monthly Anomaly')
Map

Map(center=[4.0012477311426906e-14, -155.00000000000009], controls=(WidgetControl(options=['position'], widget…

#### Create sampling grid

In [7]:
x = pd.Series(np.arange(xmin, xmax+1, res_grid)).rename('Longitude')
x[x > 180] = -(360 - x[x > 180])
y = pd.Series(np.arange(ymin, ymax+1, res_grid)).rename('Latitude')
grid = pd.DataFrame(index = x, columns = y).unstack().reset_index()[['Longitude','Latitude']]
gdf_grid = gpd.GeoDataFrame(grid, geometry = gpd.points_from_xy(grid.Longitude, grid.Latitude))
ee_grid = eeconvert.gdfToFc(gdf_grid)

#### Sample SST monthly anomolies at desired grid
*Need to update to use list comprehension instead of for loop

In [8]:
sst_grid = sst.toBands().sampleRegions(ee_grid, scale = res_sampling, properties = ['month', 'year'], geometries = True)
output = sst_grid.getInfo()
features = output['features']
features_df = pd.DataFrame()
for j in range(len(features)):
    item = features[j]
    props = item['properties']
    geom = item['geometry']
    coords = geom['coordinates']
    tmp = pd.concat([pd.DataFrame.from_dict(props, orient = 'index').transpose(), \
                     pd.DataFrame(coords).transpose()], axis = 1)
    features_df = features_df.append(tmp, ignore_index=True)

#### Clean up dataframe and export:

In [9]:
output = features_df.rename(columns = {0: 'lon', 1: 'lat'}).melt(id_vars = ['lon', 'lat'])
output[['time_index', 'feature']] = output.variable.str.split('_', expand = True)
output = output.drop('variable', axis = 1).astype({'time_index': 'int32'})

In [10]:
times = output.time_index.unique()
times.sort(axis = 0)
month_labels = pd.date_range(str(start_year)+'-01-01',str(end_year)+'-12-31', 
              freq='MS').strftime("%Y-%m").tolist()
times_df = pd.DataFrame(times, columns = ['time_index'])
times_df = pd.concat([times_df, pd.DataFrame(month_labels, columns = ['month_label'])], axis = 1)

In [11]:
output = output.merge(times_df, on = 'time_index')
output[['year', 'month']] = output.month_label.str.split('-', expand = True)
output = output.drop(['time_index', 'feature', 'month_label'], axis = 1)

In [13]:
output.head()

,lon,lat,value,year,month
0,154.016155,-29.958815,94.741935,1999,01
1,155.992449,-29.958815,65.612903,1999,01
2,157.968743,-29.958815,36.903226,1999,01
3,160.034868,-29.958815,38.903226,1999,01
4,162.011161,-29.958815,6.129032,1999,01


In [12]:
output.to_csv(file_path, index = False)